In [1]:
import pandas as pd
import json
from tqdm import tqdm
from collections import Counter

### Building 10k vocab 
10k vocab of the most frequent tokens in the {`flickr8k, flickr30k, coco`}

In [11]:
files = ['data.json', 'data30.json', 'coco.json']

freqs = {}

tokens_list = []
for file in tqdm(files, desc='Getting Token Lists'):
    df = pd.read_json(file)
    tokens_list.extend(df.tokens.to_list()) 


for tokens in tqdm(tokens_list, desc='Counting Frequencies'):
    for word in tokens:
        if word not in freqs:
            freqs[word] = 1
        else:
            freqs[word] += 1

Counting Frequencies: 100%|██████████| 811837/811837 [00:03<00:00, 259510.53it/s]


In [25]:
top_vocab_dict = dict(sorted(freqs.items(), key=lambda x: x[1], reverse=True))

In [26]:
top_vocab_dict['the']

285794

In [28]:
topk_words = list(top_vocab_dict)[:10000]

In [45]:
topk_words[:10]

['a', 'in', 'the', 'on', 'of', 'with', 'and', 'is', 'man', 'to']

In [1]:
from utils import get_topk_vocab
get_topk_vocab()[:10]

OSError: /home/kelwa/anaconda3/envs/kaggle_torch/lib/python3.7/site-packages/torchtext/_torchtext.so: undefined symbol: _ZNK3c104Type14isSubtypeOfExtESt10shared_ptrIS0_EPSo